<a href="https://colab.research.google.com/github/aidar-ms/carter/blob/master/timegan_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys

sys.version_info

sys.version_info(major=3, minor=7, micro=13, releaselevel='final', serial=0)

In [ ]:
! pip install "ydata-synthetic==0.6.1"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 2.9 MB/s 
     |████████████████████████████████| 394.5 MB 43 kB/s 
     |████████████████████████████████| 9.9 MB 50.1 MB/s 
     |████████████████████████████████| 14.8 MB 63.9 MB/s 
     |████████████████████████████████| 280 kB 77.6 MB/s 
     |████████████████████████████████| 11.6 MB 47.4 MB/s 
     |████████████████████████████████| 596 kB 71.3 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
     |████████████████████████████████| 2.9 MB 54.2 MB/s 
     |████████████████████████████████| 3.8 MB 50.9 MB/s 
     |████████████████████████████████| 462 kB 73.9 MB/s 
     |████████████████████████████████| 132 kB 75.1 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68724 sha256=0ef537244366fbba715fcea20abe571aabcd38db3fe70bbc56ee113f5046bc71
  Stored in directory: /root/.cache/pip/wheels/

In [ ]:
import tensorflow as tf

tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

# print(tf.__version__)
# print(tf.test.gpu_device_name())
# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

True

In [ ]:
from os import path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from ydata_synthetic.synthesizers import ModelParameters
from ydata_synthetic.preprocessing.timeseries import processed_stock
from ydata_synthetic.synthesizers.timeseries import TimeGAN

In [ ]:
# Specific to TimeGANs
seq_len=24
n_seq = 9
hidden_dim=24
gamma=1

noise_dim = 32
dim = 128
batch_size = 128

log_step = 100
learning_rate = 5e-4

gan_args = ModelParameters(batch_size=batch_size,
                           lr=learning_rate,
                           noise_dim=noise_dim,
                           layers_dim=dim)

In [ ]:
!ls

drive  sample_data


In [ ]:
# surface_ids = [2, 3, 4]
# user_ids_groups = [
#     [10, 11, 12, 13, 14, 15, 16],
#     [17, 18, 19, 20, 21, 22, 23],
#     [24, 25, 26, 27, 28, 29, 30]
# ]

surface_ids = [2]
user_ids_groups = [[10]]

dfs = []
for surface_id, user_ids in zip(surface_ids, user_ids_groups):
    for user_id in user_ids:
        dfs.append(
            pd.read_csv(
                f"drive/MyDrive/labeled_data/{user_id}_{surface_id}_3.csv",
                usecols=["Acc_X", "Acc_Y", "Acc_Z", "Gyr_X", "Gyr_Y", "Gyr_Z", "Mag_X", "Mag_Y", "Mag_Z"]
            )
        )


data = pd.concat(dfs, ignore_index=True)


In [ ]:
data.dropna(how="any", inplace=True)
data

,Acc_X,Acc_Y,Acc_Z,Gyr_X,Gyr_Y,Gyr_Z,Mag_X,Mag_Y,Mag_Z
0,9.647161,0.141292,-1.471709,0.166737,-0.137316,0.015382,-0.863281,-0.418701,0.190918
1,9.613439,0.072964,-1.513684,0.107456,-0.164579,0.011343,-0.863281,-0.410400,0.187012
2,9.635614,-0.058254,-1.535989,0.076312,-0.187805,0.005832,-0.855225,-0.410400,0.191162
3,9.691418,-0.112065,-1.585476,0.076973,-0.186620,0.005008,-0.863281,-0.414551,0.191162
4,9.711740,-0.107425,-1.646009,0.074935,-0.188336,0.002116,-0.865723,-0.406250,0.197510
...,...,...,...,...,...,...,...,...,...
1147,-0.408264,-4.746511,-0.658958,-0.233564,-0.307934,0.622472,-0.941650,-0.339111,0.070801
1148,-3.731499,-3.032019,-9.634005,-2.210660,-0.821853,1.221609,-0.950195,-0.334961,0.070801
1149,-8.156759,-6.697263,-15.179937,-2.739909,-2.269892,1.242874,-0.952637,-0.326660,0.092041
1150,-1.837095,-7.522774,-10.618964,-1.627467,-3.431740,0.829940,-0.950928,-0.316406,0.119385


In [ ]:
from ydata_synthetic.preprocessing.timeseries.utils import real_data_loading

data_processed = real_data_loading(data, seq_len=seq_len)

In [ ]:
print(len(data_processed), data_processed[0].shape)
data_processed

1128 (24, 9)


[array([[0.29517642, 0.64116018, 0.37164695, 0.57244084, 0.27931084,
         0.50101202, 0.32273067, 0.45314075, 0.45796101],
        [0.21974756, 0.78597213, 0.32428057, 0.57147565, 0.30888795,
         0.57149731, 0.35217947, 0.45314075, 0.41741253],
        [0.16623792, 0.78623831, 0.37450459, 0.65344598, 0.41911504,
         0.60909889, 0.32390785, 0.43563471, 0.38401924],
        [0.15489589, 0.74410344, 0.46488381, 0.64406622, 0.50982184,
         0.63587042, 0.3345073 , 0.42739632, 0.34824088],
        [0.18229944, 0.73618776, 0.53293219, 0.65381575, 0.53821258,
         0.59696334, 0.34511157, 0.40988606, 0.33303535],
        [0.2118513 , 0.70526183, 0.62620848, 0.68526873, 0.55252254,
         0.5615047 , 0.33686648, 0.38414163, 0.32766805],
        [0.25234008, 0.66037495, 0.68089274, 0.7302667 , 0.56748528,
         0.5238611 , 0.36631046, 0.37487398, 0.31246252],
        [0.31062155, 0.5857088 , 0.70439461, 0.75022079, 0.55030432,
         0.47781298, 0.38751417, 0.3666355

In [ ]:
# if path.exists('synthesizer_stock.pkl'):
#     synth = TimeGAN.load('synthesizer_stock.pkl')
# else:

model = TimeGAN(model_parameters=gan_args, hidden_dim=9, seq_len=seq_len, n_seq=n_seq, gamma=1)
model.train(data_processed, train_steps=50000)
model.save('synthesizer_stock.pkl')

Emddeding network training:  21%|██        | 10259/50000 [11:30<44:34, 14.86it/s]


KeyboardInterrupt: ignored